# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [133]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [134]:
data = pd.read_csv('../data/customer_product_sales.csv').set_index('CustomerID')

In [135]:
data.head()

,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
CustomerID,,,,,,
61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [ ]:
data_step_01 = data.groupby(['CustomerID', 'ProductName'])[['Quantity']].sum().reset_index()
data_step_01.head()

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [ ]:
data_step_02 = pd.pivot_table(data, index=['ProductName','CustomerID'], values= 'Quantity',fill_value=0)
data_step_02.tail(5)

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [ ]:
data_step_03 = data.pivot_table(values='Quantity', columns='CustomerID', index='ProductName', aggfunc='sum').fillna(0)
distances = squareform(pdist(data_step_03.T, 'euclidean'))
distances

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [ ]:
distancias = pd.DataFrame(1/(1+distances), index=data_step_03.columns, columns=data_step_03.columns)


In [ ]:
distancias.columns

Obtenemos la lista de los Customers y elegimos uno. En nuestro caso el 477


In [ ]:
customer_477_dist = distancias[477].sort_values(ascending=False)

Descartamos el mismo número porque siempre va a dar que es el más similar.  
Y obtenemos los 5 primeros `[1:6]`.  
Para obtener los elementos tenemos que obtener los índices `.index`.  
Después lo convertimos a un array con el método `to_list()`

In [ ]:
top_5 = customer_477_dist[1:6].index.to_list()
top_5

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [ ]:
step_01_df = data_step_01[data_step_01['CustomerID'].isin(top_5)]
step_01_list_top_5= step_01_df['CustomerID'].unique()
step_01_list_top_5

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [ ]:
ordered_quantity = step_01_df.groupby('ProductName')[['Quantity']].sum().sort_values('Quantity', ascending=False)
ordered_quantity

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [ ]:
not_my_products = data_step_03[data_step_03[477]==0]
merged = not_my_products.merge(ordered_quantity, left_on='ProductName', right_on='ProductName').sort_values('Quantity',ascending=False)
merged[1:6]

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [ ]:
recommendations_for_everyone = {}
keys = data.index.to_list()
keys
for key in keys:
    cust_dist = distancias[key].sort_values(ascending=False)
    top_5_cust = cust_dist[1:6].index.to_list()
    step_01_new = data_step_01[data_step_01['CustomerID'].isin(top_5_cust)]
    ord_quant = step_01_new.groupby('ProductName')[['Quantity']].sum().sort_values('Quantity', ascending=False)
    not_my_products = data_step_03[data_step_03[key]==0]
    merged = not_my_products.merge(ord_quant, left_on='ProductName', right_on='ProductName').sort_values('Quantity',ascending=False)
    values = merged[1:6]
    recommendations_for_everyone.update({key: values})
    
    


In [ ]:
recommendations_for_everyone

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [ ]:
results = pd.DataFrame.from_dict(recommendations_for_everyone, orient='index').reset_index()
results.columns=['CustomerID', 'Prod_1', 'Prod_2', 'Prod_3', 'Prod_4', 'Prod_5']
results.head()

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [ ]:
data_step_03_consine = data.pivot_table(values='Quantity', columns='CustomerID', index='ProductName', aggfunc='sum').fillna(0)
dist_consine = squareform(pdist(data_step_03_consine.T, 'consine'))
distancias_consine = pd.DataFrame(1/(1+dist_consine), index=data_step_03_consine.columns, columns=data_step_03_consine.columns)
recommendations_for_everyone_consine = {}
keys = data.index.to_list()
keys
for key in keys:
    cust_dist = distancias_consine[key].sort_values(ascending=False)
    top_5_cust = cust_dist[1:6].index.to_list()
    step_01_new = data_step_01[data_step_01['CustomerID'].isin(top_5_cust)]
    ord_quant = step_01_new.groupby('ProductName')[['Quantity']].sum().sort_values('Quantity', ascending=False)
    not_my_products = data_step_03_consine[data_step_03_consine[key]==0]
    merged = not_my_products.merge(ord_quant, left_on='ProductName', right_on='ProductName').sort_values('Quantity',ascending=False)
    values = merged[1:6]
    recommendations_for_everyone_consine.update({key: values})

In [ ]:
recommendations_for_everyone_consine